Creacion de Carpetas

In [1]:
from pathlib import Path

# Directorio base: el directorio de trabajo actual del cuaderno
base = Path.cwd()

# Nombres de las carpetas a crear
folders = ["SVG_64", "SVG_18"]

for name in folders:
    path = base / name
    path.mkdir(parents=True, exist_ok=True)  # no falla si ya existen
    print(f"Carpeta creada/existente: {path.resolve()}")


Carpeta creada/existente: C:\Users\WIRBI\Downloads\Azure_Public_Service_Icons_V22\Azure_Public_Service_Icons\Icons\blockchain\SVG_64
Carpeta creada/existente: C:\Users\WIRBI\Downloads\Azure_Public_Service_Icons_V22\Azure_Public_Service_Icons\Icons\blockchain\SVG_18


Mover .svg a carpeta SVG_18

In [2]:
from pathlib import Path
import shutil

base = Path.cwd()                # Directorio base (cuaderno actual)
dest = base / "SVG_18"           # Carpeta destino
dest.mkdir(parents=True, exist_ok=True)

# Recorre recursivamente todos los .svg
moved, skipped = 0, 0
for svg_path in base.rglob("*"):
    if svg_path.is_file() and svg_path.suffix.lower() == ".svg":
        # Evita mover archivos que ya estén dentro de SVG_18
        try:
            svg_path.relative_to(dest)
            skipped += 1
            continue
        except ValueError:
            pass

        target = dest / svg_path.name
        # Si ya existe un archivo con el mismo nombre, agrega un sufijo numérico
        if target.exists():
            stem, ext = svg_path.stem, svg_path.suffix
            i = 1
            while True:
                candidate = dest / f"{stem}_{i}{ext}"
                if not candidate.exists():
                    target = candidate
                    break
                i += 1

        shutil.move(str(svg_path), str(target))
        moved += 1

print(f"Archivos movidos: {moved}, omitidos (ya en SVG_18): {skipped}")
print(f"Destino: {dest.resolve()}")

Archivos movidos: 6, omitidos (ya en SVG_18): 6
Destino: C:\Users\WIRBI\Downloads\Azure_Public_Service_Icons_V22\Azure_Public_Service_Icons\Icons\blockchain\SVG_18


Configuración

In [3]:
# =========== NORMALIZAR SVG EXISTENTES (de 18x18 a 64x64, centrados y con padding) ===========
# Requisitos (en WSL/Ubuntu): sudo apt update && sudo apt install -y inkscape
# Nota: usamos Inkscape en WSL solo para medir bbox visual; el resultado final sigue siendo 100% vectorial.

import subprocess, shlex, tempfile, os
from pathlib import Path
import re
import xml.etree.ElementTree as ET
from PIL import Image
from time import perf_counter

# ---------- 1) CONFIGURACIÓN ----------
WIN_FOLDER   = r"C:\Users\WIRBI\Downloads\Azure_Public_Service_Icons_V22\Azure_Public_Service_Icons\Icons\blockchain"  # carpeta base
INPUT_NAME   = "SVG_18"      # <--- carpeta que contiene tus SVG actuales (18x18)
OUTPUT_NAME  = "SVG_64"      # <--- carpeta de salida con SVG normalizados a 64x64

TARGET_W     = 64            # ancho final
TARGET_H     = 64            # alto final
PADDING      = 3             # margen interno (2–4 suele ir bien)
ALPHA_CUTOFF = 80            # umbral para ignorar sombras suaves (70–90)
RENDER_PX    = 1024          # resolución de medición (suficiente para 64px)

SVG_NS   = "http://www.w3.org/2000/svg"
XLINK_NS = "http://www.w3.org/1999/xlink"
ET.register_namespace("", SVG_NS)
ET.register_namespace("xlink", XLINK_NS)

base        = Path(WIN_FOLDER)
input_dir   = base / INPUT_NAME
output_dir  = base / OUTPUT_NAME
output_dir.mkdir(parents=True, exist_ok=True)

# ---------- 2) UTILIDADES WSL / SVG ----------
def win_to_wsl_path(p: Path) -> str:
    s = str(p)
    drive = s[0].lower()
    rest = s[2:].replace("\\", "/")
    return f"/mnt/{drive}/{rest}"

def _parse_viewbox(vb: str):
    nums = [float(x) for x in re.split(r"[ ,]+", vb.strip()) if x]
    if len(nums) != 4:
        raise ValueError("viewBox inválido")
    return nums

def _float_from_unit(val: str | None):
    if val is None:
        raise ValueError("valor ausente")
    m = re.match(r"([0-9.+-eE]+)", str(val))
    if not m:
        raise ValueError(f"valor inválido: {val}")
    return float(m.group(1))

def _get_root_box(root):
    vb = root.get("viewBox")
    if vb:
        return _parse_viewbox(vb)
    w = _float_from_unit(root.get("width"))
    h = _float_from_unit(root.get("height"))
    return [0.0, 0.0, w, h]

def export_svg_to_png(svg_path: Path, png_path: Path, width_px: int):
    """Raster temporal con Inkscape (en WSL) para medir bbox visual."""
    wsl_svg = win_to_wsl_path(svg_path)
    wsl_png = win_to_wsl_path(png_path)
    cmd = (
        f'inkscape {shlex.quote(wsl_svg)} '
        f'--export-type=png '
        f'--export-filename={shlex.quote(wsl_png)} '
        f'--export-width={width_px} '
        f'--export-area-page'
    )
    subprocess.run(["wsl","bash","-lc", cmd], check=True)

def measure_bbox_with_inkscape(svg_path: Path, alpha_cutoff=80, render_px=1024):
    root = ET.parse(svg_path).getroot()
    minx, miny, vw, vh = _get_root_box(root)

    tmp_png = Path(tempfile.gettempdir()) / ("__tmp_bbox__.png")
    export_svg_to_png(svg_path, tmp_png, width_px=render_px)

    img = Image.open(tmp_png).convert("RGBA")
    out_w, out_h = img.size
    alpha = img.split()[-1]
    mask = alpha.point(lambda p: 255 if p >= alpha_cutoff else 0, mode='L')
    bbox = mask.getbbox()
    try: os.remove(tmp_png)
    except: pass

    if not bbox:
        bx_px, by_px, br_px, bb_px = 0, 0, out_w, out_h
    else:
        bx_px, by_px, br_px, bb_px = bbox

    bx = minx + (bx_px / out_w) * vw
    by = miny + (by_px / out_h) * vh
    bw = ((br_px - bx_px) / out_w) * vw
    bh = ((bb_px - by_px) / out_h) * vh
    return bx, by, bw, bh, (minx, miny, vw, vh)

# ---------- 3) NORMALIZACIÓN A 64×64 ----------
def normalize_svg_to_target(svg_path: Path, target_w=64, target_h=64,
                            padding=3, alpha_cutoff=80, render_px=1024):
    bx, by, bw, bh, (minx, miny, vw, vh) = measure_bbox_with_inkscape(
        svg_path, alpha_cutoff=alpha_cutoff, render_px=render_px
    )

    inner_w = max(target_w - 2*padding, 1)
    inner_h = max(target_h - 2*padding, 1)
    s = min(inner_w / bw if bw else 1, inner_h / bh if bh else 1)

    left_off = (target_w - s * bw) / 2
    top_off  = (target_h - s * bh) / 2

    tree = ET.parse(svg_path)
    root = tree.getroot()

    children = list(root)
    defs = [c for c in children if c.tag == f"{{{SVG_NS}}}defs"]
    others = [c for c in children if c not in defs]

    # fijar canvas final
    for attr in ["width", "height"]:
        if attr in root.attrib:
            del root.attrib[attr]
    root.set("viewBox", f"0 0 {target_w} {target_h}")
    root.set("width", str(target_w))
    root.set("height", str(target_h))
    root.set("preserveAspectRatio", "xMidYMid meet")

    # Orden correcto: translate(center) -> scale -> translate(-bbox)
    transform = (
        f"translate({left_off:.6f},{top_off:.6f}) "
        f"scale({s:.9f}) "
        f"translate({-bx:.6f},{-by:.6f})"
    )

    # reusar wrapper si existe
    wrapper = None
    for child in root:
        if child.tag == f"{{{SVG_NS}}}g" and child.get("data-normalized") == "1":
            wrapper = child
            break

    if wrapper is not None:
        wrapper.set("transform", transform)
    else:
        g = ET.Element(f"{{{SVG_NS}}}g", {
            "transform": transform,
            "data-normalized": "1"
        })
        for c in others:
            root.remove(c)
            g.append(c)
        root.append(g)

    tree.write(svg_path, encoding="utf-8", xml_declaration=True)

# ---------- 4) PROCESAR TODOS LOS SVG DE ENTRADA ----------
svgs = sorted(input_dir.glob("*.svg"))
print("Carpeta entrada:", input_dir)
print("Carpeta salida  :", output_dir)
print("SVG encontrados :", len(svgs))
if not svgs:
    raise SystemExit("No hay SVGs en la carpeta de entrada.")

t0 = perf_counter()
n = 0
for svg in svgs:
    out_path = output_dir / svg.name
    # copia original a salida y normaliza allí (no tocamos el original)
    out_path.write_bytes(svg.read_bytes())
    normalize_svg_to_target(out_path, TARGET_W, TARGET_H, PADDING, ALPHA_CUTOFF, RENDER_PX)
    n += 1
t1 = perf_counter()

print(f"\nListo. Normalizados {n} SVG a {TARGET_W}x{TARGET_H} en: {output_dir}")
print(f"Tiempo total: {t1 - t0:.2f}s")
# ============================================================================================


Carpeta entrada: C:\Users\WIRBI\Downloads\Azure_Public_Service_Icons_V22\Azure_Public_Service_Icons\Icons\blockchain\SVG_18
Carpeta salida  : C:\Users\WIRBI\Downloads\Azure_Public_Service_Icons_V22\Azure_Public_Service_Icons\Icons\blockchain\SVG_64
SVG encontrados : 6

Listo. Normalizados 6 SVG a 64x64 en: C:\Users\WIRBI\Downloads\Azure_Public_Service_Icons_V22\Azure_Public_Service_Icons\Icons\blockchain\SVG_64
Tiempo total: 7.33s


Creación de Libreria XML

In [4]:
# ==== Office 365 SVGs -> draw.io mxlibrary XML (64x64 fallback) ====
# Requisitos: solo Python estándar (json, base64) — no dependencias extra.

from pathlib import Path
import base64, json, re

# 1) CONFIGURA AQUÍ tus carpetas

INPUT_DIR  = Path(r"C:\Users\WIRBI\Downloads\Azure_Public_Service_Icons_V22\Azure_Public_Service_Icons\Icons\blockchain\SVG_64")
OUTPUT_XML = Path(r"C:\Users\WIRBI\Downloads\Azure_Public_Service_Icons_V22\Azure_Public_Service_Icons\Icons\blockchain\Azure Blockchain.xml")

# ----------------- utilidades para leer tamaño del SVG -----------------
_num_re = re.compile(r"^\s*([0-9.+-eE]+)")

def _float_from_unit(s: str | None):
    if not s: 
        return None
    m = _num_re.match(s)
    return float(m.group(1)) if m else None

def _parse_viewbox(vb: str | None):
    if not vb: 
        return None
    parts = [p for p in re.split(r"[ ,]+", vb.strip()) if p]
    if len(parts) != 4:
        return None
    _, _, w, h = map(float, parts)
    return w, h

def intrinsic_size(svg_bytes: bytes):
    """Devuelve (w, h) a partir de viewBox o width/height. Si no puede, 64x64."""
    head = svg_bytes[:8000].decode("utf-8", errors="ignore")
    m_w = re.search(r'width="([^"]+)"', head)
    m_h = re.search(r'height="([^"]+)"', head)
    m_v = re.search(r'viewBox="([^"]+)"', head)
    if m_v:
        wh = _parse_viewbox(m_v.group(1))
        if wh: 
            return wh
    w = _float_from_unit(m_w.group(1)) if m_w else None
    h = _float_from_unit(m_h.group(1)) if m_h else None
    if w and h:
        return float(w), float(h)
    # 👇 ahora el fallback es 64x64 en lugar de 512x512
    return 64.0, 64.0

# ----------------- construir la librería -----------------
items = []
svgs = sorted(INPUT_DIR.glob("*.svg"), key=lambda p: p.name.lower())

if not svgs:
    raise SystemExit(f"No encontré SVGs en: {INPUT_DIR}")

for svg_path in svgs:
    data = svg_path.read_bytes()
    w, h = intrinsic_size(data)
    b64 = base64.b64encode(data).decode("ascii")
    item = {
        "data": f"data:image/svg+xml;base64,{b64}",
        "w": int(round(w)),
        "h": int(round(h)),
        "title": svg_path.stem.replace("_", " "),
        "aspect": "fixed",
    }
    items.append(item)

json_str = json.dumps(items, ensure_ascii=False, separators=(",", ":"))
xml_text = f"<mxlibrary>{json_str}</mxlibrary>"

OUTPUT_XML.write_text(xml_text, encoding="utf-8")
print(f"OK. Generado: {OUTPUT_XML}\nEntradas: {len(items)} (ordenadas por nombre)")


OK. Generado: C:\Users\WIRBI\Downloads\Azure_Public_Service_Icons_V22\Azure_Public_Service_Icons\Icons\blockchain\Azure Blockchain.xml
Entradas: 6 (ordenadas por nombre)
